In [1]:
from ctapipe.core import Provenance
from ctapipe.utils import json2fits
from pprint import pprint

# a few nested activities


In [2]:
p = Provenance()  # note this is a singleton, so only ever one global provenence object
p.clear()
p.start_activity()
p.add_input_file("test.txt")

p.start_activity("sub")
p.add_input_file("subinput.txt")
p.add_input_file("anothersubinput.txt")
p.add_output_file("suboutput.txt")
p.finish_activity("sub")

p.start_activity("sub2")
p.add_input_file("sub2input.txt")
p.finish_activity("sub2")

p.finish_activity()

In [3]:
p.finished_activity_names

['sub', 'sub2', '/Users/kosack/anaconda/bin/python']

In [4]:
# show the input files for each activity
[ (x['activity_name'], x['input']) for x in p.provenance]

[('sub',
  [{'role': None,
    'url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/subinput.txt'},
   {'role': None,
    'url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/anothersubinput.txt'}]),
 ('sub2',
  [{'role': None,
    'url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/sub2input.txt'}]),
 ('/Users/kosack/anaconda/bin/python',
  [{'role': None,
    'url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/test.txt'}])]

In [5]:
# show start/stop/durations:
[ (x['activity_name'],x['duration_min']) for x in p.provenance]

[('sub', 0.00035000000002227694),
 ('sub2', 0.0003166666666487572),
 ('/Users/kosack/anaconda/bin/python', 0.001266666666754901)]

In [6]:
p.provenance[0]

{'activity_name': 'sub',
 'activity_uuid': 'ff8bcc05-ca48-4962-8b42-0938e7af635f',
 'duration_min': 0.00035000000002227694,
 'input': [{'role': None,
   'url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/subinput.txt'},
  {'role': None,
   'url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/anothersubinput.txt'}],
 'output': [{'role': None,
   'url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/suboutput.txt'}],
 'start': {'time_utc': '2018-05-17T16:21:13.565'},
 'status': 'sub',
 'stop': {'time_utc': '2018-05-17T16:21:13.586'},
 'system': {'arguments': ['/Users/kosack/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py',
   '-f',
   '/Users/kosack/Library/Jupyter/runtime/kernel-ed34d3b6-f237-4acc-929b-492ec388ffe1.json'],
  'ctapipe_resources_version': '0.2.14',
  'ctapipe_svc_path': None,
  'ctapipe_version': '0.6.0.post1+git7719dac',
  'environment': {'CONDA_DEFAULT_ENV': None,
   'CONDA_EXE': None,
   'CONDA_PREFIX': Non

In [7]:
print(p.as_json(indent=2))

[
  {
    "activity_name": "sub",
    "activity_uuid": "ff8bcc05-ca48-4962-8b42-0938e7af635f",
    "start": {
      "time_utc": "2018-05-17T16:21:13.565"
    },
    "stop": {
      "time_utc": "2018-05-17T16:21:13.586"
    },
    "system": {
      "ctapipe_version": "0.6.0.post1+git7719dac",
      "ctapipe_resources_version": "0.2.14",
      "ctapipe_svc_path": null,
      "executable": "/Users/kosack/anaconda/bin/python",
      "platform": {
        "architecture_bits": "64bit",
        "architecture_linkage": "",
        "machine": "x86_64",
        "processor": "i386",
        "node": "sapmcw8.local",
        "version": "Darwin Kernel Version 17.5.0: Mon Mar  5 22:24:32 PST 2018; root:xnu-4570.51.1~1/RELEASE_X86_64",
        "system": "Darwin",
        "release": "17.5.0",
        "libcver": [
          "",
          ""
        ],
        "num_cpus": 4,
        "boot_time": "2018-05-09T16:12:48.000"
      },
      "python": {
        "version_string": "3.6.2 |Anaconda custom (64-bit

## Storing this info in output files

* already this can be stored in something like an HDF5 file header, which allows hierarchies.
* Try to flatted the data so it can be stored in a key=value header in a **FITS file** (using the FITS extended keyword convention to allow >8 character keywords), or as a table 

In [8]:
def flatten_dict(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '.')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '.')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [9]:
d = dict(activity=p.provenance)

In [10]:
pprint(flatten_dict(d))

{'activity.0.activity_name': 'sub',
 'activity.0.activity_uuid': 'ff8bcc05-ca48-4962-8b42-0938e7af635f',
 'activity.0.duration_min': 0.00035000000002227694,
 'activity.0.input.0.role': None,
 'activity.0.input.0.url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/subinput.txt',
 'activity.0.input.1.role': None,
 'activity.0.input.1.url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/anothersubinput.txt',
 'activity.0.output.0.role': None,
 'activity.0.output.0.url': '/Users/kosack/Projects/CTA/Working/ctapipe/examples/notebooks/suboutput.txt',
 'activity.0.start.time_utc': '2018-05-17T16:21:13.565',
 'activity.0.status': 'sub',
 'activity.0.stop.time_utc': '2018-05-17T16:21:13.586',
 'activity.0.system.arguments.0': '/Users/kosack/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py',
 'activity.0.system.arguments.1': '-f',
 'activity.0.system.arguments.2': '/Users/kosack/Library/Jupyter/runtime/kernel-ed34d3b6-f237-4acc-929b-492ec388ffe1.json',
